<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-29 15:19:07
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.36 C
-------------------
Today PnL: 6.38 K (0.05%)
Current PnL: -23.29 L (-15.67%)
CY Booked + Current PnL: -11.85 L (-7.97%)
-------------------
Total profit:  93.69 K
Total loss:  -24.23 L
-------------------
Total Booked + Current PnL: 15.13 L (12.43%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.39%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 89.16 L (65.41%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 6.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,94.24,38.0,M-SC,2.46,83702.0,-17075.0,17167.0,0.63,-16.94,20.51,0.09,245.0,-0.99,0.61,9.97,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,3.63,189470.0,24800.0,18530.0,0.24,15.06,9.78,26.31,89.0,1.34,1.39,24.25,X40N,NTT,AC
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.86,108019.0,7515.0,21593.0,1.61,7.48,19.99,28.96,79.0,0.35,0.79,43.98,MH,ATH,METALS
42,ITC,452.00,-38.34,49.0,H-LC,1.89,199439.0,-699.0,21599.0,0.68,-0.35,10.83,10.44,4.0,-0.03,1.46,5.19,X40,NTT,FMCG
84,VOLTAS,1530.00,-1.78,48.0,H-MC,2.89,206085.0,14343.0,23411.0,2.51,7.48,11.36,19.69,99.0,0.61,1.51,14.79,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-180.68,73.0,M-SC,6.71,92394.0,-9816.0,103204.0,11.73,-9.60,111.70,91.37,208.0,-0.10,0.68,49.99,XY25,NTT,FINANCE
10,BANDHANBNK,400.00,-13.31,48.0,H-SC,4.18,217737.0,-60822.0,318266.0,4.82,-21.83,146.17,92.42,151.0,-0.19,1.60,25.83,XY24,NTT,BANKS
78,UJJIVANSFB,60.00,97.20,56.0,H-SC,10.44,124443.0,-18036.0,37557.0,4.32,-12.66,30.18,13.70,163.0,-0.48,0.91,46.88,OX40N,NTT,BANKS
52,PGHH,17973.08,-26.45,67.0,H-MC,3.19,213090.0,12270.0,56511.0,4.09,6.11,26.52,34.25,80.0,0.22,1.56,11.62,X40,ATH,FMCG
32,HINDZINC,730.22,24.68,60.0,M-LC,2.97,203123.0,-1953.0,114521.0,3.97,-0.95,56.38,54.89,52.0,-0.02,1.49,22.91,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-24.44,44.0,H-SC,17.23,202342.0,-68419.0,398371.0,-4.62,-25.27,196.88,121.86,133.0,-0.17,1.48,58.29,AR,ATH,IT
15,BSOFT,836.99,-21.38,28.0,H-SC,13.90,88243.0,-52410.0,125190.0,-4.18,-37.26,141.87,51.74,131.0,-0.42,0.65,0.63,XR,ATH,IT
26,GREENPANEL,537.00,192.64,39.0,M-SC,0.98,130784.0,-52294.0,131268.0,-2.90,-28.56,100.37,43.14,230.0,-0.40,0.96,23.02,XY24,NTT,MISC
68,STARHEALTH,761.00,6.08,41.0,H-SC,4.80,236205.0,-44918.0,177768.0,-2.61,-15.98,75.26,47.26,144.0,-0.25,1.73,26.55,XY24,NTT,INSURANCE
85,WHIRLPOOL,2270.00,-46.72,26.0,M-SC,1.30,90135.0,-1364.0,80112.0,-2.50,-1.49,88.88,86.07,223.0,-0.02,0.66,30.89,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,53.0,M-MC,10.68,226770.0,1808.0,165837.0,0.51,0.80,73.13,74.53,192.0,0.01,1.66,33.32,XY24,BTT,STEEL
37,INDIAMART,4850.92,-55.95,22.0,H-SC,9.95,123029.0,-307.0,134065.0,-1.92,-0.25,108.97,108.45,119.0,-0.00,0.90,22.05,AR,ATH,MISC
32,HINDZINC,730.22,24.68,60.0,M-LC,2.97,203123.0,-1953.0,114521.0,3.97,-0.95,56.38,54.89,52.0,-0.02,1.49,22.91,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,10.44,124443.0,-18036.0,37557.0,4.32,-12.66,30.18,13.70,163.0,-0.48,0.91,46.88,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,94.24,38.0,M-SC,2.46,83702.0,-17075.0,17167.0,0.63,-16.94,20.51,0.09,245.0,-0.99,0.61,9.97,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.53,220572.0,-49095.0,85428.0,-0.61,-18.21,38.73,13.47,138.0,-0.57,1.62,11.56,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,38.0,H-SC,1.69,141600.0,-34303.0,76322.0,-0.37,-19.50,53.90,23.89,149.0,-0.45,1.04,22.29,OX40N,NTT,CERAMICS
66,SIS,528.0,2080.83,40.0,H-SC,4.02,88000.0,-23032.0,46640.0,-0.03,-20.74,53.00,21.26,156.0,-0.49,0.65,18.90,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,53.0,M-MC,10.68,226770.0,1808.0,165837.0,0.51,0.80,73.13,74.53,192.0,0.01,1.66,33.32,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,3.63,189470.0,24800.0,18530.0,0.24,15.06,9.78,26.31,89.0,1.34,1.39,24.25,X40N,NTT,AC
84,VOLTAS,1530.00,-1.78,48.0,H-MC,2.89,206085.0,14343.0,23411.0,2.51,7.48,11.36,19.69,99.0,0.61,1.51,14.79,XY25,NTT,AC
17,CAMS,5211.76,-6.94,41.0,H-SC,1.82,105801.0,3797.0,40130.0,-0.56,3.72,37.93,43.06,122.0,0.09,0.78,21.48,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,52.0,M-LC,7.00,153950.0,13110.0,31375.0,0.98,9.31,20.38,31.59,66.0,0.42,1.13,38.26,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.86,108019.0,7515.0,21593.0,1.61,7.48,19.99,28.96,79.0,0.35,0.79,43.98,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,55.0,H-MC,8.45,182668.0,2860.0,109546.0,2.10,1.59,59.97,62.52,88.0,0.03,1.34,36.22,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.95,22.0,H-SC,9.95,123029.0,-307.0,134065.0,-1.92,-0.25,108.97,108.45,119.0,-0.00,0.90,22.05,AR,ATH,MISC
32,HINDZINC,730.22,24.68,60.0,M-LC,2.97,203123.0,-1953.0,114521.0,3.97,-0.95,56.38,54.89,52.0,-0.02,1.49,22.91,X5K,ATH,METALS
85,WHIRLPOOL,2270.00,-46.72,26.0,M-SC,1.30,90135.0,-1364.0,80112.0,-2.50,-1.49,88.88,86.07,223.0,-0.02,0.66,30.89,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-7.07,26.0,H-LC,13.18,252453.0,-14912.0,80356.0,-0.78,-5.58,31.83,24.48,48.0,-0.19,1.85,46.33,XY24,BTT,FINANCE
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.56,206096.0,-45672.0,98885.0,-0.03,-18.14,47.98,21.13,27.0,-0.46,1.51,10.86,X40,ATH,PAINTS
74,TCS,4476.75,-31.58,28.0,H-LC,12.18,272121.0,-73835.0,148687.0,-0.14,-21.34,54.64,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.03,217680.0,-24244.0,87507.0,-0.63,-10.02,40.20,26.15,8.0,-0.28,1.60,2.87,X40,ATH,IT
19,CIPLA,1795.00,-23.55,34.0,H-LC,4.22,203760.0,-740.0,42158.0,-0.81,-0.36,20.69,20.25,10.0,-0.02,1.50,8.34,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.07,46.0,H-LC,1.62,232605.0,-13546.0,92763.0,1.85,-5.50,39.88,32.18,16.0,-0.15,1.71,28.56,X200,ATH,IT
42,ITC,452.0,-38.34,49.0,H-LC,1.89,199439.0,-699.0,21599.0,0.68,-0.35,10.83,10.44,4.0,-0.03,1.46,5.19,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.47,42.0,H-LC,2.40,155270.0,-30825.0,78303.0,0.29,-16.56,50.43,25.51,15.0,-0.39,1.14,14.42,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.41,43.0,H-LC,3.93,214344.0,3978.0,24800.0,-0.26,1.89,11.57,13.68,37.0,0.16,1.57,18.73,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-23.55,34.0,H-LC,4.22,203760.0,-740.0,42158.0,-0.81,-0.36,20.69,20.25,10.0,-0.02,1.50,8.34,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,28.0,H-LC,12.18,272121.0,-73835.0,148687.0,-0.14,-21.34,54.64,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.09,46.0,H-LC,7.20,225096.0,-9048.0,57670.0,0.87,-3.86,25.62,20.77,86.0,-0.16,1.65,2.79,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.03,217680.0,-24244.0,87507.0,-0.63,-10.02,40.20,26.15,8.0,-0.28,1.60,2.87,X40,ATH,IT
83,VBL,671.64,-20.04,35.0,H-LC,8.47,288025.0,-27817.0,140470.0,1.51,-8.81,48.77,35.67,5.0,-0.20,2.11,3.83,X40N,ATH,FMCG
40,INFY,2275.00,-21.39,37.0,H-LC,8.21,306145.0,-6743.0,178421.0,-0.80,-2.16,58.28,54.87,3.0,-0.04,2.25,4.35,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.47,42.0,H-LC,2.40,155270.0,-30825.0,78303.0,0.29,-16.56,50.43,25.51,15.0,-0.39,1.14,14.42,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.34,49.0,H-LC,1.89,199439.0,-699.0,21599.0,0.68,-0.35,10.83,10.44,4.0,-0.03,1.46,5.19,X40,NTT,FMCG
19,CIPLA,1795.00,-23.55,34.0,H-LC,4.22,203760.0,-740.0,42158.0,-0.81,-0.36,20.69,20.25,10.0,-0.02,1.50,8.34,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.56,206096.0,-45672.0,98885.0,-0.03,-18.14,47.98,21.13,27.0,-0.46,1.51,10.86,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,43.0,H-LC,3.93,214344.0,3978.0,24800.0,-0.26,1.89,11.57,13.68,37.0,0.16,1.57,18.73,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,28.0,H-LC,12.18,272121.0,-73835.0,148687.0,-0.14,-21.34,54.64,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-21.39,37.0,H-LC,8.21,306145.0,-6743.0,178421.0,-0.80,-2.16,58.28,54.87,3.0,-0.04,2.25,4.35,X40,BTT,IT
42,ITC,452.00,-38.34,49.0,H-LC,1.89,199439.0,-699.0,21599.0,0.68,-0.35,10.83,10.44,4.0,-0.03,1.46,5.19,X40,NTT,FMCG
83,VBL,671.64,-20.04,35.0,H-LC,8.47,288025.0,-27817.0,140470.0,1.51,-8.81,48.77,35.67,5.0,-0.20,2.11,3.83,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,46.0,H-LC,7.68,249941.0,-11678.0,130894.0,0.65,-4.46,52.37,45.57,7.0,-0.09,1.83,6.35,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,42.0,L-SC,40.24,78374.0,-35175.0,75223.0,0.96,-30.98,95.98,35.27,268.0,-0.47,0.58,91.82,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,36.0,H-SC,12.30,90270.0,-7710.0,29311.0,-0.58,-7.87,32.47,22.05,152.0,-0.26,0.66,31.21,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-180.68,73.0,M-SC,6.71,92394.0,-9816.0,103204.0,11.73,-9.60,111.70,91.37,208.0,-0.10,0.68,49.99,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.86,108019.0,7515.0,21593.0,1.61,7.48,19.99,28.96,79.0,0.35,0.79,43.98,MH,ATH,METALS
78,UJJIVANSFB,60.00,97.20,56.0,H-SC,10.44,124443.0,-18036.0,37557.0,4.32,-12.66,30.18,13.70,163.0,-0.48,0.91,46.88,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1167.94,63.0,M-SC,13.92,172529.0,-8614.0,54088.0,0.74,-4.76,31.35,25.10,235.0,-0.16,1.27,38.27,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-180.68,73.0,M-SC,6.71,92394.0,-9816.0,103204.0,11.73,-9.60,111.70,91.37,208.0,-0.10,0.68,49.99,XY25,NTT,FINANCE
60,SAIL,228.00,43.54,53.0,M-MC,10.68,226770.0,1808.0,165837.0,0.51,0.80,73.13,74.53,192.0,0.01,1.66,33.32,XY24,BTT,STEEL
7,ATULAUTO,844.00,3505.41,46.0,M-SC,7.40,112383.0,-34985.0,86805.0,-0.36,-23.74,77.24,35.16,236.0,-0.40,0.82,14.79,XY24,NTT,AUTO
32,HINDZINC,730.22,24.68,60.0,M-LC,2.97,203123.0,-1953.0,114521.0,3.97,-0.95,56.38,54.89,52.0,-0.02,1.49,22.91,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.22
1,25,43.75
2,50,75.05


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.68
LC    32.54
MC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.28
X40      14.82
X40N     11.46
XR       11.07
XY25     10.25
AR        8.20
OX40N     7.90
X200      1.71
MH        1.61
X5K       1.49
SR        1.19
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.22
H-LC    25.99
H-MC    20.83
M-SC    13.00
M-LC     5.50
M-MC     2.63
L-SC     1.46
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.15,-6.42,41.55
IT,12.72,-21.35,87.52
FINANCE,10.14,-18.73,70.52
MISC,6.97,-21.01,88.28
BANKS,6.28,-16.56,77.88
PAINTS,5.64,-20.12,38.02
ELECTRICAL,5.49,-9.99,48.58
INSURANCE,3.75,-8.29,48.77
AC,3.61,1.67,14.79


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3253517.0,22
XR,1401747.0,14
AR,1213822.0,9
X40,829069.0,10
X40N,644294.0,9
OX40N,570371.0,10
XY25,442261.0,7
SR,258664.0,2
X5K,114521.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3812949.0,29
M-SC,1472264.0,17
H-LC,1277765.0,15
H-MC,1268097.0,15
M-LC,399749.0,4
M-MC,319285.0,2
L-SC,263197.0,3
L-MC,61459.0,1
L-LC,40820.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1250587.0      6
M-SC       XY24       845272.0      7
H-SC       AR         832321.0      5
           XR         815923.0      7
H-MC       XY24       584592.0      4
H-LC       X40        535099.0      5
M-MC       XY24       319285.0      2
H-SC       X40N       319131.0      4
           OX40N      263360.0      4
           SR         258664.0      2
H-LC       X40N       240298.0      3
H-MC       X40        215655.0      4
H-LC       AR         209197.0      2
H-MC       XY25       178185.0      2
L-SC       XR         173823.0      2
M-LC       XY24       173425.0      2
M-SC       AR         172304.0      2
           OX40N      143976.0      4
           XR         129193.0      2
H-LC       XY25       120052.0      3
M-LC       X5K        114521.0      1
           XR         111803.0      1
H-MC       XR         109546.0      1
M-SC       XY25       103204.0      1
H-LC       X200        92763.0      1
L-SC       OX40N       89374.0      1
H-MC       X40N        84865.0      2
H-LC       XY24        80356.0      1
M-SC       X40         78315.0      1
H-MC       OX40N       73661.0      1
H-SC       MH          72963.0      1
L-MC       XR          61459.0      1
L-LC       XY25        40820.0      1
H-MC       MH          21593.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
